In [2]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path

import ijson

## Import JSON 

In [3]:
# Dates
start = time.time()

data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)
dates = [datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y') for prefix, event, value in data if prefix == 'created_at']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")

elapsed_time:1083.1176714897156[sec]


In [4]:
# Tweets
start = time.time()
data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)
tweets = [value for prefix, event, value in data if prefix == 'text']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")

elapsed_time:1233.47380900383[sec]


## Check the date of dates and tweets

In [21]:
print(len(dates) == len(tweets))

True


## Make Dataframe from json data

In [5]:
datescopy, twt = dates.copy(), tweets.copy()
# date, twt = l_date[:100], l_text[:100]

In [7]:
data = np.vstack([datescopy, twt]).T

df = pd.DataFrame(data, columns=['date', 'tweet'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['date'] = df['date'].dt.date

# RT_flag: True if tweet containt 'RT', otherwise False
df['RT_flag'] = df['tweet'].str.contains('RT')

# df

# divide df into one with and without RT
dfrt = df[df['RT_flag'] == True]
dfor = df[df['RT_flag'] == False]

## Connect all JSON data for the future work

In [129]:
# jsons = ['part_v003_o001_r_00000.json',..., 'part_v003_o001_r_00000.json']
# def load(jsonfile):
#     return dates, tweets

# def makedf(dates, tweets):
#     return dfrt, dfor

# for json in jsons:
#     dates, tweets = load(json)
#     dfrt, dfor = makedf(dates, tweets)
#     dfrt, dfor = pd.concat([df1, df2], axis= 1)

IndentationError: expected an indented block (<ipython-input-129-6662b3f87dbc>, line 4)

## Use Dataframe withour RT

In [101]:
# check dataframe's nan value
dfor.isnull().values.any() # FalseならOK

False

## Read keywords csv file

In [8]:
dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
dfw.columns

Index(['types', 'file_name', 'orignal_form', 'English_translation'], dtype='object')

### This time we use 'C' types wrods as an example

In [18]:
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
xd = {}
xd['T'] = 'file_name.str.contains("T")'
xd['D'] = 'file_name.str.contains("D")'
xd['A'] = 'file_name.str.contains("A")'
xd['V'] = 'file_name.str.contains("V")'
xd['F'] = 'file_name.str.contains("F")'
xd['C'] = 'file_name.str.contains("C")'


In [21]:
xd[filetypes[0]]

'file_name.str.contains("T")'

In [22]:
'''
filetypes: words' meta kind
filenames: columns correspended to keyword e.g. ['C1', 'C2',..'C35'] 
words_x: words dataframe of x(= C,D,...)
'''
# Drop nan and False in order to use query 
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
dfw = dfw.replace(booleanDictionary)
dfw = dfw.dropna(how='all')

filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
filetype = 'C'
filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
a = 'file_name.str.contains("C")'
dfw_c = dfw.query(xd['T'], engine='python')
print('types:',set(dfw.types.tolist()))

'''
dfw_x:  dfw_t = dfw.query('file_name.str.contains("T")', engine='python')
        dfw_d = dfw.query('file_name.str.contains("D")', engine='python')
        dfw_a = dfw.query('file_name.str.contains("A")', engine='python')
        dfw_v = dfw.query('file_name.str.contains("V")', engine='python')
        dfw_f = dfw.query('file_name.str.contains("F")', engine='python')
        dfw_c = dfw.query('file_name.str.contains("C")', engine='python')
'''

types: {nan, '#Confusion', '#Depression', '#Vigor', '#Anger', '#Fatigue', '#Tension'}


'\ndfw_x:  dfw_t = dfw.query(\'file_name.str.contains("T")\', engine=\'python\')\n        dfw_d = dfw.query(\'file_name.str.contains("D")\', engine=\'python\')\n        dfw_a = dfw.query(\'file_name.str.contains("A")\', engine=\'python\')\n        dfw_v = dfw.query(\'file_name.str.contains("V")\', engine=\'python\')\n        dfw_f = dfw.query(\'file_name.str.contains("F")\', engine=\'python\')\n        dfw_c = dfw.query(\'file_name.str.contains("C")\', engine=\'python\')\n'

## Find tweets which conatins keywords using dfor and words

In [105]:
keywords_jp = dfw_c.orignal_form.tolist()

dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords_jp))]
dfor_c.head()

''' You can find by English words using following codes instead
keywords_en = dfw_c.English_translation.tolist()
dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords_jp))]
'''

,date,tweet,year,month,day,hour,RT_flag
1,2020-02-29,今夜は久しぶりに \nお月様を見れましたが、\n少し雲がかかって\nボンヤリ月でした\n☁🌛...,2020,2,29,14,False
3,2020-02-29,不二先輩の誕生日で言いたいことたくさんあるけど、上手くまとまらないね😭😭,2020,2,29,14,False
106,2020-02-29,10ヶ月の間ほんとにお世話になりました&lt;(_ _)&gt; ミャーさんやごごさんの...,2020,2,29,14,False
124,2020-02-29,@k7ya ２のオープニングバトルは、全滅しないと先に進まないのですよー。\n何かやりようが...,2020,2,29,14,False
133,2020-02-29,推しのソロを聞いた時に想った事をいつか言葉に出来たらって思うけど、語彙力無さすぎて全然まとま...,2020,2,29,14,False


## Add columns(['C1'], ['C2'],...) to dfor_c

In [106]:
'''
keywords_jp: keyword exist in tweet
filenames: columns correspended to keyword e.g. ['C1', 'C2',..'C35'] 
'''
start = time.time()

for col, key in zip(filenames, keywords_jp):
    l = []
    for row in dfor_c.itertuples():
        if key in row.tweet:
            l.append(1)
        else:
            l.append(0)
    dfor_c[col] = l
    
print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")

# save to csv 
# df_c_query.to_csv('c.csv')

# dfor_c.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


elapsed_time:31.51155686378479[sec]


In [107]:
# dfor_c.head()

,date,tweet,year,month,day,hour,RT_flag,C1,C2,C3,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
1,2020-02-29,今夜は久しぶりに \nお月様を見れましたが、\n少し雲がかかって\nボンヤリ月でした\n☁🌛...,2020,2,29,14,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-02-29,不二先輩の誕生日で言いたいことたくさんあるけど、上手くまとまらないね😭😭,2020,2,29,14,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106,2020-02-29,10ヶ月の間ほんとにお世話になりました&lt;(_ _)&gt; ミャーさんやごごさんの...,2020,2,29,14,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124,2020-02-29,@k7ya ２のオープニングバトルは、全滅しないと先に進まないのですよー。\n何かやりようが...,2020,2,29,14,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133,2020-02-29,推しのソロを聞いた時に想った事をいつか言葉に出来たらって思うけど、語彙力無さすぎて全然まとま...,2020,2,29,14,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Group by date & make rows (dict)

In [108]:
uni_dates = dfor_c['date'].tolist()
uni_dates = sorted(set(uni_dates), key=uni_dates.index)

In [109]:
rows = []
for date in uni_dates:
    d = {}
    d['date'] = str(date)
    for col in filenames:
        _df = dfor_c.groupby('date').get_group(date)
        d[col] = _df[col].sum()
    rows.append(d)

## Make dataframes of each word types and add rows

In [110]:
cols = filenames.copy()
cols.insert(0, 'date')

In [135]:
dft = pd.DataFrame(columns=cols)
for row in rows:
    dft = dft.append(row, ignore_index=True)

# dft

In [123]:
# save to csv
# outname = 'c_original.csv'
# outdir = '../results/jp/day'

# if not os.path.exists(outdir):
#         os.makedirs(outdir)
    
# fullname = os.path.join(outdir, outname)    

# dft.to_csv(fullname)

## FINISH 

### 以下のコードは、あるfiletypeに関する20のjsonfileから得られるDataFrame(csv)の結果を一つにまとめる物です

In [172]:
dfs = [df1, df2, df3, ..., df20]
for i, df in enumerate(dfs):
    if i == 0:
        dfnew = df
    else:
        dfnew = pd.concat([dfnew, df], axis=0)

In [170]:
df1 = dft[:1]
df2 = dft[:1]
df3 = dft[2:3]


In [173]:
dfnew

,date,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,2020-02-29,1980,2624,1253,522,1118,1090,984,446,745,...,1266,430,729,321,151,53,105,166,235,261
0,2020-02-29,1980,2624,1253,522,1118,1090,984,446,745,...,1266,430,729,321,151,53,105,166,235,261
2,2020-02-27,2504,3018,1163,790,1326,1586,1234,592,1062,...,1701,1026,858,369,218,74,148,239,288,342
